## Import

In [1]:
import numpy as np
import random
import pandas as pd
import warnings; warnings.filterwarnings("ignore")

## Read Data

In [2]:
data_path = '../data/'

pdde = pd.read_csv(data_path+"DC_LPOINT_PDDE.csv", parse_dates=['date', 'de_dt'])       # [DC] 유통사 상품 구매 내역      

## Generate Columns

#### $\blacktriangleright$ 거주지 내 구매
EDA를 통해 거주지 내 구매는 온라인 구매로 전환될 때 탄소배출 감소에 도움이 될 것이라 확인하였다. 이에 거주지 내 구매 여부를 나타내는 열을 만들어 ESG Score를 산출한다.

In [3]:
pdde['거주지내구매'] = (pdde['home'] == pdde['store']).astype('i')

#### $\blacktriangleright$ 친환경 상품
롯데지주의 지속가능경영보고서를 바탕으로 저탄소 상품, 중고거래상품, 친환경 포장재 상품 등을 통한 2040년 탄소중립을 목표로 한다. 임의로 해당 상품 여부열을 만들어 ESG Score를 산출한다.

In [4]:
pdde['저탄소상품'] = [random.choice([0,0,0,0,1]) for i in range(pdde.shape[0])]

In [5]:
pdde['중고거래상품'] = [random.choice([0,0,0,0,0,0,0,0,1]) for i in range(pdde.shape[0])]

In [6]:
pdde['친환경포장재'] = [random.choice([0,0,0,0,1]) for i in range(pdde.shape[0])]

#### $\blacktriangleright$ ESG 협력사 지원
2021년에 발표된 K-ESG 평가기준은 S분야의 ESG 협력사 경영, 지원을 거론하며 상생경영을 강조하였다. ESG 협력사의 제품임을 나타내는 임의의 열을 생성하여 ESG Score를 산출한다.

In [7]:
pdde['ESG협력사제품'] = [random.choice([0,0,0,0,1]) for i in range(pdde.shape[0])]

## Calculate ESG Score

In [8]:
ESG = pdde.drop_duplicates(['cust','cop_c','chnl_dv','date'])[['cust','cop_c','chnl_dv','date']].reset_index(drop=True)
ESG

,cust,cop_c,chnl_dv,date
0,M430112881,A01,1,2021-01-01 10:00:00
1,M646853852,A01,1,2021-07-16 20:00:00
2,M646853852,A01,1,2021-09-18 20:00:00
3,M284446967,A01,1,2021-08-06 14:00:00
4,M401266932,A01,1,2021-01-23 16:00:00
...,...,...,...,...
1058101,M570817832,A05,2,2021-07-05 11:00:00
1058102,M152568682,A06,2,2021-02-03 21:00:00
1058103,M118790272,A06,2,2021-03-30 09:00:00
1058104,M897387507,A06,2,2021-03-07 10:00:00


In [9]:
ESG = pd.merge(ESG,
               pdde.query('chnl_dv==2').groupby(['cust','cop_c','chnl_dv','date'])['buy_ct'].sum().rename('온라인상품수').reset_index(),
               on=['cust','cop_c','chnl_dv','date'], how='outer')\
      .merge(pdde.query('거주지내구매==1').groupby(['cust','cop_c','chnl_dv','date'])['buy_ct'].sum().rename('거주지내상품수').reset_index(),
               on=['cust','cop_c','chnl_dv','date'], how='outer')\
      .merge(pdde.query('저탄소상품==1').groupby(['cust','cop_c','chnl_dv','date'])['buy_am'].sum().rename('저탄소상품액').reset_index(),
               on=['cust','cop_c','chnl_dv','date'], how='outer')\
      .merge(pdde.query('중고거래상품==1').groupby(['cust','cop_c','chnl_dv','date'])['buy_am'].sum().rename('중고거래상품액').reset_index(),
             on=['cust','cop_c','chnl_dv','date'], how='outer')\
      .merge(pdde.query('친환경포장재==1').groupby(['cust','cop_c','chnl_dv','date'])['buy_am'].sum().rename('친환경포장재상품액').reset_index(),
             on=['cust','cop_c','chnl_dv','date'], how='outer')\
      .merge(pdde.query('ESG협력사제품==1').groupby(['cust','cop_c','chnl_dv','date'])['buy_am'].sum().rename('ESG협력사상품액').reset_index(),
             on=['cust','cop_c','chnl_dv','date'], how='outer')
ESG.fillna(0, inplace=True)
ESG

,cust,cop_c,chnl_dv,date,온라인상품수,거주지내상품수,저탄소상품액,중고거래상품액,친환경포장재상품액,ESG협력사상품액
0,M430112881,A01,1,2021-01-01 10:00:00,0.0,0.0,15000.0,28900.0,0.0,19000.0
1,M646853852,A01,1,2021-07-16 20:00:00,0.0,1.0,0.0,0.0,0.0,20000.0
2,M646853852,A01,1,2021-09-18 20:00:00,0.0,1.0,0.0,0.0,0.0,0.0
3,M284446967,A01,1,2021-08-06 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0
4,M401266932,A01,1,2021-01-23 16:00:00,0.0,0.0,0.0,0.0,49000.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1058101,M570817832,A05,2,2021-07-05 11:00:00,1.0,0.0,0.0,0.0,0.0,0.0
1058102,M152568682,A06,2,2021-02-03 21:00:00,1.0,0.0,0.0,19800.0,0.0,0.0
1058103,M118790272,A06,2,2021-03-30 09:00:00,1.0,0.0,0.0,0.0,0.0,0.0
1058104,M897387507,A06,2,2021-03-07 10:00:00,1.0,0.0,908000.0,0.0,0.0,0.0


In [10]:
ESG_SCORE = pd.concat([ESG[['cust','cop_c','chnl_dv','date']],
                       (ESG['온라인상품수'] * 700).rename('온라인상품수_마일리지'),
                       (ESG['거주지내상품수'] * -700).rename('거주지내구매_감점'),
                       (ESG[[i for i in ESG.columns if '액' in i]].sum(axis=1) * 0.01).rename('상품액_마일리지')], axis=1)

ESG_SCORE = pd.merge(ESG_SCORE,
                     ESG_SCORE.set_index(['cust','cop_c','chnl_dv','date']).sum(axis=1).rename('ESG_score').reset_index(),
                     on=['cust','cop_c','chnl_dv','date'], how='outer')
ESG_SCORE

,cust,cop_c,chnl_dv,date,온라인상품수_마일리지,거주지내구매_감점,상품액_마일리지,ESG_score
0,M430112881,A01,1,2021-01-01 10:00:00,0.0,-0.0,629.0,629.0
1,M646853852,A01,1,2021-07-16 20:00:00,0.0,-700.0,200.0,-500.0
2,M646853852,A01,1,2021-09-18 20:00:00,0.0,-700.0,0.0,-700.0
3,M284446967,A01,1,2021-08-06 14:00:00,0.0,-0.0,0.0,0.0
4,M401266932,A01,1,2021-01-23 16:00:00,0.0,-0.0,490.0,490.0
...,...,...,...,...,...,...,...,...
1058101,M570817832,A05,2,2021-07-05 11:00:00,700.0,-0.0,0.0,700.0
1058102,M152568682,A06,2,2021-02-03 21:00:00,700.0,-0.0,198.0,898.0
1058103,M118790272,A06,2,2021-03-30 09:00:00,700.0,-0.0,0.0,700.0
1058104,M897387507,A06,2,2021-03-07 10:00:00,700.0,-0.0,9080.0,9780.0


## Marketing

In [12]:
# 고객 로그인 시 마일리지 출력
CUST, DAY = 'M533330698', '2021-02-08'
CUST_SCORE = ESG_SCORE.query('cust == @CUST & date < @DAY').ESG_score.sum()
print(f'{CUST} 고객님! ESG 마일리지를 {CUST_SCORE} 포인트를 쓸 수 있습니다.')

M533330698 고객님! ESG 마일리지를 48733.899999999994 포인트를 쓸 수 있습니다.


## Save data

In [13]:
ESG_SCORE.to_csv(data_path + "ESG_SCORE.csv")